# 一般化した数独の問題作成&ソルバー by pulp
- 数独はニコリの登録商標です
- ナンプレとも呼ばれます
- pulpを使って整数計画問題を解く練習のために作ったnotebookです

## ソルバー
- 与えられた問題の解を全て出力する

## 問題作成
- 方法1: 空の盤面にランダムに数字を入れていき, ソルバーで解が一意かどうか判定しながらやる
- 方法2: 埋まった盤面からランダムに数字を抜いていき, ソルバーで解が一意かどうか判定しながらやる
    - 埋まった盤面を作る必要がある

## 一般化
- 普通の数独(9x9盤面)の正方形ブロック(3x3)を$m \times n$にした
- 盤面のサイズは $mn \times mn$

Prologバージョン
- https://github.com/birdwatcherYT/SudokuProlog/blob/master/sudoku.pro


## 整数計画問題
### pulp
インストール方法
- Anacondaの場合
    - `conda install -c conda-forge pulp`
- pipの場合
    - `pip install pulp`

使い方:
- https://docs.pyq.jp/python/math_opt/pulp.html

In [2]:
import numpy as np
import pulp

In [4]:
class Sudoku:
    """数独クラス"""
    def __init__(self, m, n):
        """考える小さいブロックのサイズ(m, n): 普通の数独ならm=3, n=3"""
        self.m = m
        self.n = n
        # 入る数字の最大数
        self.max_num = m * n
        # ループ用
        self.r = range(self.max_num)

    def init(self, problem):
        """与えられた問題を解くための初期化用関数
         problem: 0が空欄をあらわすnp.ndarray(int)
        """
        # 最適化問題
        self.sudoku = pulp.LpProblem("sudoku", pulp.LpMinimize)
        # max_num x max_numの盤面の数を表す変数
        self.board = np.array(
            [
                [
                    [
                        pulp.LpVariable(f"board_{i}_{j}_{k}", cat="Binary")
                        for k in self.r
                    ]
                    for j in self.r
                ]
                for i in self.r
            ]
        )
        # 数字はどれか一つ
        for i in self.r:
            for j in self.r:
                self.sudoku += pulp.lpSum(self.board[i,j,:]) == 1
        # 行に同じ数字が入ってはいけない
        for i in self.r:
            for k in self.r:
                self.sudoku += pulp.lpSum(self.board[i,:,k]) == 1
        # 列に同じ数字が入ってはいけない
        for j in self.r:
            for k in self.r:
                self.sudoku += pulp.lpSum(self.board[:,j,k]) == 1
        # 正方形の枠に同じ数字が入ってはいけない
        for i in range(0, self.max_num, self.m):
            for j in range(0, self.max_num, self.n):
                for k in self.r:
                    self.sudoku += pulp.lpSum(self.board[i:i+self.m, j:j+self.n, k]) == 1
        # 初期値をいれる
        for i in self.r:
            for j in self.r:
                if problem[i, j]:# 空欄でないとき
                    self.sudoku += self.board[i, j, problem[i, j]-1] == 1

    def solve(self, problem):
        """ソルバー. 全ての解が順番に得られる
         problem: 0が空欄をあらわすnp.ndarray(int)
        """
        self.init(problem)
        while True:
            self.sudoku.solve()
            if pulp.LpStatus[self.sudoku.status]=="Optimal":
                answer = np.vectorize(pulp.value)(self.board)
                answer = np.where(answer)[2].reshape(self.max_num,self.max_num)+1
                yield answer
                # 割り当てられた変数から最低でも1つ外すようにすることで別の解を探すようにする
                self.sudoku += pulp.lpSum([
                    self.board[i, j, k]
                    for i in self.r
                    for j in self.r
                    for k in self.r
                    if pulp.value(self.board[i, j, k]) == 1
                ]) <= self.max_num * self.max_num-1
            else:
                break

    def is_unique(self, problem):
        """解が一意かどうか判定
         True: 一意
         False: 一意でない
         None: 解なし
        """
        cnt = 0
        for ans in self.solve(problem):
            cnt+=1
            # 一意でない
            if cnt>=2:
                return False
        # 一意
        if cnt==1:
            return True
        # 解なし
        return None

    def create_problem_from_empty(self, try_num):
        """問題作成 (空の盤面にランダムに数字を入れていくバージョン)
         実行速度遅い.
        """
        # 空の盤面作成
        problem = np.zeros((self.max_num, self.max_num), dtype=int)
        # 埋めていく
        for _ in range(try_num):
            print(">", end="")
            while True:
                i, j = np.random.randint(self.max_num, size=2)
                if not problem[i, j]:
                    break
            problem[i, j] = np.random.randint(self.max_num) + 1
            is_unique = self.is_unique(problem)
            if is_unique is None:
                problem[i, j] = 0
                continue
            if is_unique:
                print("")
                return problem
        return None

    def create_problem_from_fill(self, try_num, level=1):
        """問題作成 (埋まった状態から穴をあけていくバージョン)
         最初に答えの盤面を作る必要がある
        """
        # 空の盤面作成
        problem = np.zeros((self.max_num, self.max_num), dtype=int)
        # 盤面を全て埋める
        if not self.__fill_rows(problem, 0, try_num):
#         if not fill_rows(self.m, self.n, self.max_num, problem, 0, try_num):
            return None
        # 盤面に穴をあける
        cnt = 0
        while True:
            print(">", end="")
            while True:
                i, j = np.random.randint(self.max_num, size=2)
                if problem[i, j]:
                    break
            tmp = problem[i, j]
            problem[i, j] = 0
            if not self.is_unique(problem):
                problem[i, j] = tmp
                cnt += 1
                if cnt >= level:
                    break
        print("")
        return problem

    def print_board(self, problem):
        """盤面の表示"""
        for i in self.r:
            if i%self.m==0:
                print("----"*self.max_num+"-"*(self.m+1))
            for j in self.r:
                if j%self.n==0:
                    print("|", end="")
                print(f"{problem[i,j]:>3} " if problem[i,j] else "    ", end="")
            print("|")
        print("----"*self.max_num+"-"*(self.m+1))

    def __fill_rows(self, problem, i, try_num):
        """i行以降をランダムに埋める再帰関数 (埋まった状態の盤面作成用ヒューリスティック)"""
        if i >= self.max_num:
            return True
        for _ in range(try_num):
            if not self.__fill_vals(problem, i, 0, try_num):
                continue
            if self.__fill_rows(problem, i+1, try_num):
                return True
        return False

    def __fill_vals(self, problem, i, j, try_num):
        """i行目のj列以降をランダムに埋める再帰関数 (埋まった状態の盤面作成用ヒューリスティック)"""
        if j >= self.max_num: # 最後の列まで埋まったら
            return True
        for _ in range(try_num):
            problem[i, j] = np.random.randint(self.max_num)+1
            # 重複判定
            row = problem[i, :j+1]
            column = problem[:i+1, j]
            block = problem[i//m*m:i+1, j//n*n:(j//n+1)*n]
            zero_exist = (block == 0).any()
            if np.unique(row).shape[0] != row.shape[0] or \
                np.unique(column).shape[0] != column.shape[0] or \
                np.unique(block).shape[0]-zero_exist != (block != 0).sum():
                continue # 重複したら別の数字をトライ
            if self.__fill_vals(problem, i, j+1, try_num): #次の値を埋めに行く
                return True
        problem[i, j] = 0
        return False

In [5]:
# m, n = 1, 2
# m, n = 2, 2
# m, n = 2, 3 # 3s
m, n = 3, 3 # 10s
# m, n = 2, 5 # 
# m, n = 2, 6 # 
# m, n = 3, 4 # 25s
# m, n = 4, 4 # 1min
# m, n = 4, 5 # 3min

sudoku = Sudoku(m, n)

In [6]:
%%time
# 埋まった盤面から穴を開けていく方法
problem = sudoku.create_problem_from_fill(m*n, level=1)
sudoku.print_board(problem)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
----------------------------------------
|      2   5 |  9   7     |  3   6   8 |
|      7     |      6   8 |          5 |
|      6   1 |  2   5     |          4 |
----------------------------------------
|            |  7   9     |  1       6 |
|  7   9   6 |      3   5 |          2 |
|  3         |  6   8   4 |          9 |
----------------------------------------
|  6   4     |      1     |  5       7 |
|          9 |          7 |      8     |
|  1   5   7 |  8   2   6 |  4   9   3 |
----------------------------------------
Wall time: 6.15 s


In [7]:
%%time
# 空欄の盤面を埋めていく方法
# m*n>10からきつくなる
problem = sudoku.create_problem_from_empty(1000)
sudoku.print_board(problem)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
----------------------------------------
|  5         |  2   9     |  4         |
|  4         |      3     |  5         |
|      2     |      7     |      6   8 |
----------------------------------------
|      6     |      4   1 |            |
|  7         |  8   5     |      1   2 |
|  8       3 |      2     |  9         |
----------------------------------------
|      7     |      1   9 |      5     |
|  6         |            |  1       4 |
|      5     |      6     |            |
----------------------------------------
Wall time: 19 s


In [8]:
# 作った問題を解く
for ans in sudoku.solve(problem):
    sudoku.print_board(ans)

----------------------------------------
|  5   3   6 |  2   9   8 |  4   7   1 |
|  4   8   7 |  1   3   6 |  5   2   9 |
|  9   2   1 |  5   7   4 |  3   6   8 |
----------------------------------------
|  2   6   5 |  9   4   1 |  7   8   3 |
|  7   4   9 |  8   5   3 |  6   1   2 |
|  8   1   3 |  6   2   7 |  9   4   5 |
----------------------------------------
|  3   7   8 |  4   1   9 |  2   5   6 |
|  6   9   2 |  7   8   5 |  1   3   4 |
|  1   5   4 |  3   6   2 |  8   9   7 |
----------------------------------------


In [9]:
# 盤面埋めるプロセスをjitで高速化したバージョン
# 実装が汚くなるので一旦したまで避けてある (コメントアウトして使ってない)
from numba import njit

@njit
def fill_rows(m,n,max_num, problem, i, try_num):
    if i >= max_num:
        return True
    for _ in range(try_num):
        if not fill_vals(m,n,max_num,problem, i, 0, try_num):
            continue
        if fill_rows(m,n,max_num,problem, i+1, try_num):
            return True
    return False
@njit
def fill_vals(m,n,max_num, problem, i, j, try_num):
    if j >= max_num:
        return True
    for _ in range(try_num):
        problem[i,j] = np.random.randint(max_num)+1
        # 重複判定
        row = problem[i,:j+1]
        column = problem[:i+1,j]
        block = problem[i//m*m:i+1, j//n*n:(j//n+1)*n]
        zero_exist = (block == 0).any()
        if np.unique(row).shape[0] != row.shape[0] or \
            np.unique(column).shape[0] != column.shape[0] or \
            np.unique(block).shape[0]-zero_exist != (block != 0).sum():
            continue
        if fill_vals(m,n,max_num,problem, i, j+1, try_num):
            return True
    problem[i,j] = 0
    return False